In [1]:
from sunpy.map import Map
import matplotlib.pyplot as plt
import numpy as np
import astropy.units as u
import pandas as pd
from glob import glob
import re
from astropy.coordinates import SkyCoord
import copy

In [109]:
flare_location_df = pd.read_csv('/Users/hannahcollier/Documents/solo/data/stx_catalogue_pipeline/OneDrive_1_01-06-2023/flare_locations_firstpass.csv')


In [110]:
flare_location_df['flare location test'] = 'not tested'

In [111]:
flare_location_df

,flare_id,att_in,GOES_flux,start_UTC,peak_UTC,end_UTC,4-10 keV,10-15 keV,number_available_files,available_file_request_IDs,...,aux_paths,pixel_paths,Request IDs_y,x,y,dx,dy,maxx,maxy,flare location test
0,2111012338,False,4.519747e-06,2021-11-01T23:35:18.784,2021-11-01 23:38:42.784,2021-11-02T00:06:42.787,23551,2943,2,"[2111010026, 2111010028]",...,./aux_data/solo_L2_stix-aux-ephemeris_20211101...,./pixel_data/solo_L1_stix-sci-xray-cpd_2021110...,2.111010e+09,-37.934402,281.747772,0.464911,0.314686,-40.944420,280.761749,not tested
1,2111032112,False,5.239915e-06,2021-11-03T21:01:03.064,2021-11-03 21:12:11.065,2021-11-03T22:23:19.070,14847,799,2,"[2111030013, 2111030014]",...,./aux_data/solo_L2_stix-aux-ephemeris_20211103...,./pixel_data/solo_L1_stix-sci-xray-cpd_2021110...,2.111030e+09,1051.242554,-478.418854,0.331888,0.408078,1048.811279,-484.066711,not tested
2,2111041050,False,3.542749e-06,2021-11-04T10:34:19.144,2021-11-04 10:50:55.146,2021-11-04T11:35:15.151,13823,799,1,[2111040016],...,./aux_data/solo_L2_stix-aux-ephemeris_20211104...,./pixel_data/solo_L1_stix-sci-xray-cpd_2021110...,2.111040e+09,-1002.274597,-584.970642,0.377174,0.343974,-1004.046326,-590.953003,not tested
3,2112042356,False,7.139402e-06,2021-12-04T23:27:16.028,2021-12-04 23:56:11.931,2021-12-05T00:40:03.936,17407,991,1,[2112040041],...,./aux_data/solo_L2_stix-aux-ephemeris_20211204...,./pixel_data/solo_L1_stix-sci-xray-cpd_2021120...,2.112040e+09,885.645813,-435.147766,0.336415,0.304578,882.094910,-436.200775,not tested
4,2112050108,False,3.924801e-06,2021-12-05T01:05:03.938,2021-12-05 01:08:39.939,2021-12-05T01:59:43.944,12799,1215,1,[2112050017],...,./aux_data/solo_L2_stix-aux-ephemeris_20211205...,./pixel_data/solo_L1_stix-sci-xray-cpd_2021120...,2.112050e+09,860.298767,-463.965607,0.300284,0.203155,857.787415,-465.240662,not tested
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1238,2304080124,False,2.900991e-05,2023-04-08T01:18:32.006,2023-04-08 01:24:36.006,2023-04-08T01:43:08.008,15871,991,2,"[2304087134, 2304086842]",...,./aux_data/solo_L2_stix-aux-ephemeris_20230408...,./pixel_data/solo_L1_stix-sci-xray-cpd_2023040...,2.304087e+09,2967.391113,1311.596558,0.191149,0.177668,2956.703125,1306.830078,not tested
1239,2304090559,False,9.095766e-06,2023-04-09T05:50:28.193,2023-04-09 05:59:40.194,2023-04-09T06:50:24.199,63487,4863,1,[2304099032],...,./aux_data/solo_L2_stix-aux-ephemeris_20230409...,./pixel_data/solo_L1_stix-sci-xray-cpd_2023040...,2.304099e+09,-3048.334961,-1152.925415,2.839967,1.165050,-3047.255859,-1175.843262,not tested
1240,2304201600,True,3.389966e-06,2023-04-20T15:55:52.674,2023-04-20 16:00:52.675,2023-04-20T16:41:36.679,1507327,253951,2,"[2304201471, 2304204748]",...,./aux_data/solo_L2_stix-aux-ephemeris_20230420...,./pixel_data/solo_L1_stix-sci-xray-cpd_2023042...,2.304201e+09,-102.206856,-583.731567,0.239517,0.194938,-105.692093,-594.518005,not tested
1241,2304210330,True,9.182903e-07,2023-04-21T03:24:40.748,2023-04-21 03:30:12.749,2023-04-21T03:58:44.752,172031,10751,1,[2304216828],...,./aux_data/solo_L2_stix-aux-ephemeris_20230421...,./pixel_data/solo_L1_stix-sci-xray-cpd_2023042...,2.304217e+09,-455.575012,254.233704,0.220470,0.281166,-467.970795,246.984573,not tested


In [112]:
%matplotlib inline
filenamelist = glob('/Users/hannahcollier/Documents/solo/data/stx_catalogue_pipeline/bp_maps/*.fits')
filenamelist.sort()
bp_maps = Map(filenamelist)
bp_maps_copy = copy.deepcopy(bp_maps)

for i in range(len(bp_maps)):
    m = re.search('bp_nat_map-(.+?).fits', filenamelist[i])
    req_id = m.group(1)
    if len(flare_location_df[flare_location_df['Request IDs_x'] == int(req_id)]) != 0:
        loc_x = flare_location_df[flare_location_df['Request IDs_x'] == int(req_id)]['x'][flare_location_df[flare_location_df['Request IDs_x'] == int(req_id)]['x'].index[-1]]
        loc_y = flare_location_df[flare_location_df['Request IDs_x'] == int(req_id)]['y'][flare_location_df[flare_location_df['Request IDs_x'] == int(req_id)]['y'].index[-1]]
              
        pix_x = int(bp_maps[i].world_to_pixel(SkyCoord(loc_x*u.arcsec, loc_y*u.arcsec, frame=bp_maps[i].coordinate_frame))[1].value)
        pix_y = int(bp_maps[i].world_to_pixel(SkyCoord(loc_x*u.arcsec, loc_y*u.arcsec, frame=bp_maps[i].coordinate_frame))[0].value)
        
        original_max_val = bp_maps[i].data[pix_x,pix_y]

        bp_maps_copy[i].data[pix_x-20:pix_x+20,pix_y-20:pix_y+20] = 0
        
        new_max_val = bp_maps_copy[i].data.max()
        
        if new_max_val < 0.9*original_max_val:
            print('location looks good')
            flare_location_df[flare_location_df['Request IDs_x'] == int(req_id)] = flare_location_df.loc[flare_location_df['Request IDs_x'] == int(req_id)].replace('not tested', 'pass')

        else:
            print('not sure about this location')
            flare_location_df[flare_location_df['Request IDs_x'] == int(req_id)] = flare_location_df.loc[flare_location_df['Request IDs_x'] == int(req_id)].replace('not tested', 'fail')



location looks good
location looks good
location looks good
location looks good
location looks good
location looks good
location looks good
location looks good
location looks good
location looks good
location looks good
location looks good
location looks good
location looks good
location looks good
location looks good
location looks good
location looks good
location looks good
location looks good
location looks good
location looks good
location looks good
location looks good
location looks good
location looks good
location looks good
location looks good
location looks good
location looks good
location looks good
location looks good
location looks good
location looks good
location looks good
location looks good
location looks good
location looks good
location looks good
location looks good
location looks good
location looks good
location looks good
location looks good
location looks good
location looks good
location looks good
location looks good
location looks good
location looks good


location looks good
location looks good
location looks good
location looks good
location looks good
location looks good
location looks good
location looks good
location looks good
not sure about this location
location looks good
location looks good
location looks good
location looks good
location looks good
location looks good
location looks good
location looks good
location looks good
location looks good
location looks good
location looks good
location looks good
location looks good
location looks good
location looks good
location looks good
location looks good
location looks good
location looks good
location looks good
location looks good
location looks good
location looks good
location looks good
location looks good
location looks good
location looks good
location looks good
location looks good
location looks good
location looks good
location looks good
location looks good
location looks good
location looks good
not sure about this location
location looks good
location looks good
lo

location looks good
location looks good
not sure about this location
location looks good
location looks good
location looks good
location looks good
location looks good
location looks good
location looks good
location looks good
location looks good
location looks good
location looks good
location looks good
location looks good
location looks good
location looks good
location looks good
location looks good
location looks good
location looks good
location looks good
location looks good
location looks good
location looks good
location looks good
location looks good
location looks good
location looks good
location looks good
location looks good
location looks good
location looks good
location looks good
location looks good
location looks good
location looks good
location looks good
location looks good
location looks good
location looks good
location looks good
location looks good
location looks good
location looks good
location looks good
location looks good
location looks good
location lo

In [113]:
flare_location_df

,flare_id,att_in,GOES_flux,start_UTC,peak_UTC,end_UTC,4-10 keV,10-15 keV,number_available_files,available_file_request_IDs,...,aux_paths,pixel_paths,Request IDs_y,x,y,dx,dy,maxx,maxy,flare location test
0,2111012338,False,4.519747e-06,2021-11-01T23:35:18.784,2021-11-01 23:38:42.784,2021-11-02T00:06:42.787,23551,2943,2,"[2111010026, 2111010028]",...,./aux_data/solo_L2_stix-aux-ephemeris_20211101...,./pixel_data/solo_L1_stix-sci-xray-cpd_2021110...,2.111010e+09,-37.934402,281.747772,0.464911,0.314686,-40.944420,280.761749,pass
1,2111032112,False,5.239915e-06,2021-11-03T21:01:03.064,2021-11-03 21:12:11.065,2021-11-03T22:23:19.070,14847,799,2,"[2111030013, 2111030014]",...,./aux_data/solo_L2_stix-aux-ephemeris_20211103...,./pixel_data/solo_L1_stix-sci-xray-cpd_2021110...,2.111030e+09,1051.242554,-478.418854,0.331888,0.408078,1048.811279,-484.066711,pass
2,2111041050,False,3.542749e-06,2021-11-04T10:34:19.144,2021-11-04 10:50:55.146,2021-11-04T11:35:15.151,13823,799,1,[2111040016],...,./aux_data/solo_L2_stix-aux-ephemeris_20211104...,./pixel_data/solo_L1_stix-sci-xray-cpd_2021110...,2.111040e+09,-1002.274597,-584.970642,0.377174,0.343974,-1004.046326,-590.953003,pass
3,2112042356,False,7.139402e-06,2021-12-04T23:27:16.028,2021-12-04 23:56:11.931,2021-12-05T00:40:03.936,17407,991,1,[2112040041],...,./aux_data/solo_L2_stix-aux-ephemeris_20211204...,./pixel_data/solo_L1_stix-sci-xray-cpd_2021120...,2.112040e+09,885.645813,-435.147766,0.336415,0.304578,882.094910,-436.200775,pass
4,2112050108,False,3.924801e-06,2021-12-05T01:05:03.938,2021-12-05 01:08:39.939,2021-12-05T01:59:43.944,12799,1215,1,[2112050017],...,./aux_data/solo_L2_stix-aux-ephemeris_20211205...,./pixel_data/solo_L1_stix-sci-xray-cpd_2021120...,2.112050e+09,860.298767,-463.965607,0.300284,0.203155,857.787415,-465.240662,pass
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1238,2304080124,False,2.900991e-05,2023-04-08T01:18:32.006,2023-04-08 01:24:36.006,2023-04-08T01:43:08.008,15871,991,2,"[2304087134, 2304086842]",...,./aux_data/solo_L2_stix-aux-ephemeris_20230408...,./pixel_data/solo_L1_stix-sci-xray-cpd_2023040...,2.304087e+09,2967.391113,1311.596558,0.191149,0.177668,2956.703125,1306.830078,pass
1239,2304090559,False,9.095766e-06,2023-04-09T05:50:28.193,2023-04-09 05:59:40.194,2023-04-09T06:50:24.199,63487,4863,1,[2304099032],...,./aux_data/solo_L2_stix-aux-ephemeris_20230409...,./pixel_data/solo_L1_stix-sci-xray-cpd_2023040...,2.304099e+09,-3048.334961,-1152.925415,2.839967,1.165050,-3047.255859,-1175.843262,fail
1240,2304201600,True,3.389966e-06,2023-04-20T15:55:52.674,2023-04-20 16:00:52.675,2023-04-20T16:41:36.679,1507327,253951,2,"[2304201471, 2304204748]",...,./aux_data/solo_L2_stix-aux-ephemeris_20230420...,./pixel_data/solo_L1_stix-sci-xray-cpd_2023042...,2.304201e+09,-102.206856,-583.731567,0.239517,0.194938,-105.692093,-594.518005,pass
1241,2304210330,True,9.182903e-07,2023-04-21T03:24:40.748,2023-04-21 03:30:12.749,2023-04-21T03:58:44.752,172031,10751,1,[2304216828],...,./aux_data/solo_L2_stix-aux-ephemeris_20230421...,./pixel_data/solo_L1_stix-sci-xray-cpd_2023042...,2.304217e+09,-455.575012,254.233704,0.220470,0.281166,-467.970795,246.984573,pass


In [114]:
len(flare_location_df[flare_location_df['flare location test']=='pass'])/len(flare_location_df) 

0.9493161705551086

In [115]:
len(flare_location_df[flare_location_df['flare location test']=='fail'])/len(flare_location_df) 

0.050683829444891394

In [116]:
len(flare_location_df[flare_location_df['flare location test']=='not tested'])/len(flare_location_df) 

0.0

In [147]:
flare_location_df.to_csv('/Users/hannahcollier/Documents/solo/data/stx_catalogue_pipeline/flare_location_df_first_pass_duplicates.csv')


In [ ]:
for i in range(len(bp_maps)):
    m = re.search('bp_nat_map-(.+?).fits', filenamelist[i])
    req_id = m.group(1)
    if len(flare_location_df[flare_location_df['Request IDs_x'] == int(req_id)]) != 0:
        loc_x = flare_location_df[flare_location_df['Request IDs_x'] == int(req_id)]['x'][flare_location_df[flare_location_df['Request IDs_x'] == int(req_id)]['x'].index[-1]]
        loc_y = flare_location_df[flare_location_df['Request IDs_x'] == int(req_id)]['y'][flare_location_df[flare_location_df['Request IDs_x'] == int(req_id)]['y'].index[-1]]
        
        result = flare_location_df[flare_location_df['Request IDs_x'] == int(req_id)]['flare location test'].values[0]
        
        fig = plt.figure(figsize=(12, 4))
        ax = fig.add_subplot(1,2,1, projection=bp_maps[i])
        bp_maps[i].plot(cmap='viridis')
        ax.plot_coord(SkyCoord(loc_x*u.arcsec, loc_y*u.arcsec, frame=bp_maps[i].coordinate_frame),  "x", color='red')
        
        ax = fig.add_subplot(1,2,2, projection=bp_maps_copy[i])
        bp_maps_copy[i].plot(cmap='viridis')
        ax1.plot_coord(SkyCoord(loc_x*u.arcsec, loc_y*u.arcsec, frame=bp_maps_copy[i].coordinate_frame), "x", color='red')
        plt.suptitle(f'{result}')
        plt.savefig(f'bp_maps_edited_to_check_max/comparison_bp_map_{i}.png', dpi=300)
        
        plt.show()